In [1]:
# declaraciones de librerías
using DataStructures

# Cycle-cancelling

# Llamando a una rutina escrita en C (scaling algorithm de Goldberg)

En ocasiones un procedimiento o rutina ya existe en otro lenguaje (en nuestra área típicamente en C o en Fortran) y uno quiere hacer uso del mismo en vez de entrar en detalles de implementación.

Este es el caso de los algoritmos de cost scaling para max-flow min-cost. Es un algoritmo difícil de implementar que requiere mucho cuidado para obtener implementaciones verdaderamente eficientes (esto es, implementaciones que superen los resultados obtenidos por algoritmos más simples o por el uso de Cplex). 

Goldberg publicó una implementación de su algoritmo que utlizaremos aquí (saltándonos ligeramente las leyes de copyright ya que el código no es estrictamente libre y consideraré que se trata de "abandonware").  

Nosotros únicamente llamaremos a esa implementación 

# ¿Qué se nos ha quedado en el tintero?